In [5]:
"""
Model Diagnostic and Fix Script
================================
Diagnoses why the model is not learning meaningful patterns and provides
solutions to improve risk classification performance.

Part of: Policy Risk Inference from Simulated Reports
Author: William V. Fullerton
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


class ModelDiagnostic:
    """Diagnose and fix model performance issues."""
    
    def __init__(self, df):
        """Initialize diagnostic with dataframe."""
        self.df = df.copy()
        print(f"Initialized diagnostic with {len(self.df)} reports")
        
    def run_full_diagnostic(self, target_col='risk_label'):
        """Run comprehensive diagnostic on data and features."""
        print("\n" + "="*70)
        print("MODEL DIAGNOSTIC REPORT")
        print("="*70)
        
        # Find target column
        if target_col not in self.df.columns:
            label_cols = [col for col in self.df.columns if 'label' in col.lower() or 'target' in col.lower()]
            if label_cols:
                target_col = label_cols[0]
                print(f"Using target column: '{target_col}'")
            else:
                print("ERROR: No target column found!")
                print(f"Available columns: {list(self.df.columns)}")
                return None
        
        # Get features
        risk_feature_cols = [col for col in self.df.columns if col.startswith('risk_')]
        
        if not risk_feature_cols:
            print("ERROR: No risk features found!")
            print(f"Available columns: {list(self.df.columns)}")
            print("\nDid you run script 02 (language analysis) first?")
            return None
        
        print(f"\nFeatures being used: {risk_feature_cols}")
        print(f"Number of features: {len(risk_feature_cols)}")
        
        y = self.df[target_col].values
        X = self.df[risk_feature_cols].values
        
        # DIAGNOSTIC 1: Class Distribution
        print("\n" + "-"*70)
        print("DIAGNOSTIC 1: CLASS IMBALANCE")
        print("-"*70)
        
        unique, counts = np.unique(y, return_counts=True)
        for label, count in zip(unique, counts):
            pct = 100 * count / len(y)
            print(f"  Class {label}: {count:5d} samples ({pct:.2f}%)")
        
        imbalance_ratio = counts.max() / counts.min()
        print(f"\n  Imbalance Ratio: {imbalance_ratio:.1f}:1")
        
        if imbalance_ratio > 20:
            print("  ⚠ SEVERE CLASS IMBALANCE DETECTED")
            print("  → This is likely causing the model to predict only the majority class")
        
        # DIAGNOSTIC 2: Feature Statistics by Class
        print("\n" + "-"*70)
        print("DIAGNOSTIC 2: FEATURE DISCRIMINATIVE POWER")
        print("-"*70)
        
        for feature in risk_feature_cols:
            class_0_mean = self.df[self.df[target_col] == 0][feature].mean()
            class_1_mean = self.df[self.df[target_col] == 1][feature].mean()
            
            class_0_std = self.df[self.df[target_col] == 0][feature].std()
            class_1_std = self.df[self.df[target_col] == 1][feature].std()
            
            # Calculate separation (effect size)
            pooled_std = np.sqrt((class_0_std**2 + class_1_std**2) / 2)
            effect_size = abs(class_1_mean - class_0_mean) / (pooled_std + 1e-10)
            
            print(f"\n  {feature}:")
            print(f"    Low-risk mean:  {class_0_mean:.4f} (std: {class_0_std:.4f})")
            print(f"    High-risk mean: {class_1_mean:.4f} (std: {class_1_std:.4f})")
            print(f"    Effect size (Cohen's d): {effect_size:.4f}")
            
            if effect_size < 0.2:
                print(f"    ⚠ WEAK: Features barely differ between classes")
            elif effect_size < 0.5:
                print(f"    → SMALL: Some separation but limited")
            elif effect_size < 0.8:
                print(f"    ✓ MEDIUM: Reasonable discriminative power")
            else:
                print(f"    ✓✓ LARGE: Strong discriminative power")
        
        # DIAGNOSTIC 3: Feature Variance
        print("\n" + "-"*70)
        print("DIAGNOSTIC 3: FEATURE VARIANCE")
        print("-"*70)
        
        for feature in risk_feature_cols:
            variance = self.df[feature].var()
            print(f"  {feature}: variance = {variance:.6f}")
            
            if variance < 0.001:
                print(f"    ⚠ Near-zero variance - feature may not be informative")
        
        # DIAGNOSTIC 4: Feature Correlation
        print("\n" + "-"*70)
        print("DIAGNOSTIC 4: FEATURE-TARGET CORRELATION")
        print("-"*70)
        
        for feature in risk_feature_cols:
            correlation = self.df[feature].corr(self.df[target_col])
            print(f"  {feature}: r = {correlation:.4f}")
            
            if abs(correlation) < 0.05:
                print(f"    ⚠ Very weak correlation with target")
        
        # DIAGNOSTIC 5: Baseline Model Check
        print("\n" + "-"*70)
        print("DIAGNOSTIC 5: BASELINE MODEL PERFORMANCE")
        print("-"*70)
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Simple logistic regression
        model = LogisticRegression(random_state=42, max_iter=1000)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
        
        print("\n  Confusion Matrix:")
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
        
        # Check if model is predicting only one class
        unique_preds = np.unique(y_pred)
        if len(unique_preds) == 1:
            print(f"\n  ⚠⚠ CRITICAL: Model only predicts class {unique_preds[0]}")
            print("  → Model has collapsed to majority class prediction")
        
        # Check probability distribution
        print(f"\n  Predicted probability statistics:")
        print(f"    Min:  {y_proba.min():.6f}")
        print(f"    Max:  {y_proba.max():.6f}")
        print(f"    Mean: {y_proba.mean():.6f}")
        print(f"    Std:  {y_proba.std():.6f}")
        
        if y_proba.std() < 0.01:
            print(f"    ⚠ Very low variance in predictions")
            print(f"    → Model is not confident in distinguishing classes")
        
        # SUMMARY AND RECOMMENDATIONS
        print("\n" + "="*70)
        print("DIAGNOSTIC SUMMARY & RECOMMENDATIONS")
        print("="*70)
        
        print("\nIdentified Issues:")
        
        if imbalance_ratio > 20:
            print("\n  1. SEVERE CLASS IMBALANCE")
            print("     → Use SMOTE or class weighting")
            print("     → Consider cost-sensitive learning")
        
        max_effect_size = max([
            abs(self.df[self.df[target_col] == 1][f].mean() - 
                self.df[self.df[target_col] == 0][f].mean()) / 
            (self.df[f].std() + 1e-10)
            for f in risk_feature_cols
        ])
        
        if max_effect_size < 0.5:
            print("\n  2. WEAK FEATURE SEPARATION")
            print("     → Features don't strongly distinguish between classes")
            print("     → Need better feature engineering")
            print("     → Consider using TF-IDF directly or adding more features")
        
        if len(unique_preds) == 1:
            print("\n  3. MODEL COLLAPSE")
            print("     → Model only predicting majority class")
            print("     → Apply solutions below immediately")
        
        print("\n" + "="*70)
        
        return {
            'imbalance_ratio': imbalance_ratio,
            'max_effect_size': max_effect_size,
            'model_collapsed': len(unique_preds) == 1,
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test
        }


class ModelFixer:
    """Apply fixes to improve model performance."""
    
    def __init__(self, diagnostic_results):
        """Initialize with diagnostic results."""
        self.results = diagnostic_results
        self.X_train = diagnostic_results['X_train']
        self.X_test = diagnostic_results['X_test']
        self.y_train = diagnostic_results['y_train']
        self.y_test = diagnostic_results['y_test']
        self.models = {}
        
    def fix_with_smote(self):
        """Solution 1: Oversample minority class with SMOTE."""
        print("\n" + "="*70)
        print("SOLUTION 1: SMOTE (Synthetic Minority Oversampling)")
        print("="*70)
        
        try:
            # Apply SMOTE
            smote = SMOTE(random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(self.X_train, self.y_train)
            
            print(f"\nOriginal training set: {len(self.y_train)} samples")
            print(f"After SMOTE: {len(y_train_smote)} samples")
            
            unique, counts = np.unique(y_train_smote, return_counts=True)
            for label, count in zip(unique, counts):
                print(f"  Class {label}: {count} samples")
            
            # Train model
            model = LogisticRegression(random_state=42, max_iter=1000)
            model.fit(X_train_smote, y_train_smote)
            
            # Evaluate
            y_pred = model.predict(self.X_test)
            y_proba = model.predict_proba(self.X_test)[:, 1]
            
            print("\nResults:")
            print(classification_report(self.y_test, y_pred, 
                                       target_names=['Low Risk', 'High Risk']))
            
            cm = confusion_matrix(self.y_test, y_pred)
            print("\nConfusion Matrix:")
            print(cm)
            
            if len(np.unique(y_pred)) > 1:
                auc = roc_auc_score(self.y_test, y_proba)
                print(f"\nROC-AUC: {auc:.3f}")
                print("✓ Model is now predicting both classes!")
            
            self.models['smote'] = {
                'model': model,
                'y_pred': y_pred,
                'y_proba': y_proba,
                'method': 'SMOTE Oversampling'
            }
            
        except Exception as e:
            print(f"Error with SMOTE: {e}")
            print("Note: SMOTE requires at least 2 minority samples")
        
        return self
    
    def fix_with_class_weights(self):
        """Solution 2: Use class weights to penalize errors on minority class."""
        print("\n" + "="*70)
        print("SOLUTION 2: CLASS WEIGHTS")
        print("="*70)
        
        # Calculate class weights
        unique, counts = np.unique(self.y_train, return_counts=True)
        total = len(self.y_train)
        class_weights = {label: total / (len(unique) * count) 
                        for label, count in zip(unique, counts)}
        
        print(f"\nCalculated class weights:")
        for label, weight in class_weights.items():
            print(f"  Class {label}: {weight:.2f}")
        
        # Train with class weights
        model = LogisticRegression(
            class_weight=class_weights,
            random_state=42,
            max_iter=1000
        )
        model.fit(self.X_train, self.y_train)
        
        # Evaluate
        y_pred = model.predict(self.X_test)
        y_proba = model.predict_proba(self.X_test)[:, 1]
        
        print("\nResults:")
        print(classification_report(self.y_test, y_pred,
                                   target_names=['Low Risk', 'High Risk']))
        
        cm = confusion_matrix(self.y_test, y_pred)
        print("\nConfusion Matrix:")
        print(cm)
        
        if len(np.unique(y_pred)) > 1:
            auc = roc_auc_score(self.y_test, y_proba)
            print(f"\nROC-AUC: {auc:.3f}")
            print("✓ Model is now predicting both classes!")
        
        self.models['class_weights'] = {
            'model': model,
            'y_pred': y_pred,
            'y_proba': y_proba,
            'method': 'Class Weights'
        }
        
        return self
    
    def fix_with_undersampling(self):
        """Solution 3: Undersample majority class."""
        print("\n" + "="*70)
        print("SOLUTION 3: RANDOM UNDERSAMPLING")
        print("="*70)
        
        # Apply undersampling
        rus = RandomUnderSampler(random_state=42)
        X_train_under, y_train_under = rus.fit_resample(self.X_train, self.y_train)
        
        print(f"\nOriginal training set: {len(self.y_train)} samples")
        print(f"After undersampling: {len(y_train_under)} samples")
        
        unique, counts = np.unique(y_train_under, return_counts=True)
        for label, count in zip(unique, counts):
            print(f"  Class {label}: {count} samples")
        
        # Train model
        model = LogisticRegression(random_state=42, max_iter=1000)
        model.fit(X_train_under, y_train_under)
        
        # Evaluate
        y_pred = model.predict(self.X_test)
        y_proba = model.predict_proba(self.X_test)[:, 1]
        
        print("\nResults:")
        print(classification_report(self.y_test, y_pred,
                                   target_names=['Low Risk', 'High Risk']))
        
        cm = confusion_matrix(self.y_test, y_pred)
        print("\nConfusion Matrix:")
        print(cm)
        
        if len(np.unique(y_pred)) > 1:
            auc = roc_auc_score(self.y_test, y_proba)
            print(f"\nROC-AUC: {auc:.3f}")
            print("✓ Model is now predicting both classes!")
        
        self.models['undersampling'] = {
            'model': model,
            'y_pred': y_pred,
            'y_proba': y_proba,
            'method': 'Random Undersampling'
        }
        
        return self
    
    def fix_with_gradient_boosting(self):
        """Solution 4: Use Gradient Boosting with scale_pos_weight."""
        print("\n" + "="*70)
        print("SOLUTION 4: GRADIENT BOOSTING WITH WEIGHTED CLASSES")
        print("="*70)
        
        # Calculate scale_pos_weight
        unique, counts = np.unique(self.y_train, return_counts=True)
        scale_pos_weight = counts[0] / counts[1] if len(counts) > 1 else 1.0
        
        print(f"\nScale pos weight: {scale_pos_weight:.2f}")
        
        # Train model
        model = GradientBoostingClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            random_state=42
        )
        
        # Manual sample weighting
        sample_weights = np.ones(len(self.y_train))
        sample_weights[self.y_train == 1] = scale_pos_weight
        
        model.fit(self.X_train, self.y_train, sample_weight=sample_weights)
        
        # Evaluate
        y_pred = model.predict(self.X_test)
        y_proba = model.predict_proba(self.X_test)[:, 1]
        
        print("\nResults:")
        print(classification_report(self.y_test, y_pred,
                                   target_names=['Low Risk', 'High Risk']))
        
        cm = confusion_matrix(self.y_test, y_pred)
        print("\nConfusion Matrix:")
        print(cm)
        
        if len(np.unique(y_pred)) > 1:
            auc = roc_auc_score(self.y_test, y_proba)
            print(f"\nROC-AUC: {auc:.3f}")
            print("✓ Model is now predicting both classes!")
        
        self.models['gradient_boosting'] = {
            'model': model,
            'y_pred': y_pred,
            'y_proba': y_proba,
            'method': 'Gradient Boosting (Weighted)'
        }
        
        return self
    
    def compare_all_solutions(self):
        """Compare all solutions side-by-side."""
        print("\n" + "="*70)
        print("SOLUTION COMPARISON")
        print("="*70)
        
        comparison_data = []
        
        for method_name, results in self.models.items():
            y_pred = results['y_pred']
            y_proba = results['y_proba']
            
            # Calculate metrics
            cm = confusion_matrix(self.y_test, y_pred)
            tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (cm[0, 0], 0, 0, 0)
            
            accuracy = (tp + tn) / len(self.y_test)
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            if len(np.unique(y_pred)) > 1:
                auc = roc_auc_score(self.y_test, y_proba)
            else:
                auc = 0.5
            
            comparison_data.append({
                'Method': results['method'],
                'Accuracy': f'{accuracy:.3f}',
                'Precision': f'{precision:.3f}',
                'Recall': f'{recall:.3f}',
                'F1-Score': f'{f1:.3f}',
                'ROC-AUC': f'{auc:.3f}',
                'FP': fp,
                'FN': fn
            })
        
        comparison_df = pd.DataFrame(comparison_data)
        print("\n" + comparison_df.to_string(index=False))
        
        print("\n" + "="*70)
        print("RECOMMENDATION")
        print("="*70)
        print("\nBased on your use case, choose:")
        print("  • SMOTE: Best overall balance, creates synthetic examples")
        print("  • Class Weights: Fast, no data augmentation needed")
        print("  • Undersampling: When you have plenty of majority class data")
        print("  • Gradient Boosting: Often best performance for complex patterns")
        
        return comparison_df


def main():
    """Run diagnostic and apply fixes."""
    print("=" * 70)
    print("MODEL DIAGNOSTIC & FIX PIPELINE")
    print("=" * 70)
    
    # Load data (output from script 02)
    data_path = 'data/processed/reports_with_features_and_labels.csv'
    print(f"\nLoading data from: {data_path}")
    
    if not os.path.exists(data_path):
        print(f"ERROR: File not found: {data_path}")
        print("\nPlease run script 02 (language analysis) first to generate this file.")
        return
    
    df = pd.read_csv(data_path)
    print(f"Loaded {len(df)} rows with columns: {list(df.columns)}")
    
    # Run diagnostic
    diagnostic = ModelDiagnostic(df)
    results = diagnostic.run_full_diagnostic()
    
    # Check if diagnostic succeeded
    if results is None:
        print("\n" + "="*70)
        print("DIAGNOSTIC FAILED")
        print("="*70)
        print("\nPlease check:")
        print("  1. Did you run script 02 first?")
        print("  2. Does your data have a target/label column?")
        print("  3. Does your data have risk features (columns starting with 'risk_')?")
        return
    
    # Apply fixes
    print("\n" + "="*70)
    print("APPLYING FIXES")
    print("="*70)
    
    fixer = ModelFixer(results)
    fixer.fix_with_smote()
    fixer.fix_with_class_weights()
    fixer.fix_with_undersampling()
    fixer.fix_with_gradient_boosting()
    
    # Compare solutions
    comparison_df = fixer.compare_all_solutions()
    
    # Save comparison
    os.makedirs('reports', exist_ok=True)
    comparison_df.to_csv('reports/solution_comparison.csv', index=False)
    print("\nSaved comparison to: reports/solution_comparison.csv")
    
    print("\n" + "="*70)
    print("DIAGNOSTIC COMPLETE")
    print("="*70)


if __name__ == "__main__":
    main()

MODEL DIAGNOSTIC & FIX PIPELINE

Loading data from: data/processed/reports_with_features_and_labels.csv
Loaded 3000 rows with columns: ['id', 'timestamp', 'style', 'topic', 'sentiment', 'load_factor', 'agents', 'capacity', 'text', 'style_id', 'topic_id', 'sentiment_id', 'cleaned_text', 'risk_high_severity_count', 'risk_violation_count', 'risk_financial_count', 'risk_temporal_count', 'risk_density', 'text_length', 'composite_risk_score', 'risk_label', 'risk_label_from_sentiment', 'risk_label_from_load', 'risk_label_synthetic']
Initialized diagnostic with 3000 reports

MODEL DIAGNOSTIC REPORT

Features being used: ['risk_high_severity_count', 'risk_violation_count', 'risk_financial_count', 'risk_temporal_count', 'risk_density', 'risk_label', 'risk_label_from_sentiment', 'risk_label_from_load', 'risk_label_synthetic']
Number of features: 9

----------------------------------------------------------------------
DIAGNOSTIC 1: CLASS IMBALANCE
-------------------------------------------------

C:\Users\rfull\AppData\Roaming\Python\Python312\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\rfull\AppData\Roaming\Python\Python312\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


  risk_high_severity_count: r = nan
  risk_violation_count: r = nan
  risk_financial_count: r = nan
  risk_temporal_count: r = nan
  risk_density: r = nan
  risk_label: r = nan
  risk_label_from_sentiment: r = nan
  risk_label_from_load: r = nan
  risk_label_synthetic: r = nan

----------------------------------------------------------------------
DIAGNOSTIC 5: BASELINE MODEL PERFORMANCE
----------------------------------------------------------------------


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values